<html>
    <h1 style='color:pink;'>MOTIVATION</h1>
</html>

![FETAL](https://media.istockphoto.com/vectors/pregnancy-fertilization-related-vector-id975895412?k=6&m=975895412&s=612x612&w=0&h=L-uCk_0_OJsVAedKwijXtRfxJ0NkZaqOsfX20K9Hq-s=)

* **Cardiotocography can be used to monitor a baby's heart rate and a mother's contractions while the baby is in the uterus**

* **CTG is used both before birth and during labour, to monitor the baby for any signs of distress. By looking at various different aspects of the baby's heart rate, doctors can see the condition of a baby**

* **The vast majority of these deaths (94%) occurred in low-resource settings, and most could have been prevented**

* **I have worked on data and trying to predict the well being of baby**

<html>
    <h1 style='color:pink;'>TABLE OF CONTENTS</h1>
</html>

* **ANALYSIS**


* **PREDICTING WITHOUT REMOVING ANY FEATURES**
   * **RANDOM FOREST CLASSIFIER**
   * **LIGHT GBM**
   
* **ANALYSING WITH MUTUAL INFORMATION**
   * **PLOTTING MI SCORES**
   * **VISUALIZING IMPORTANT FEATURES**
   * **PREDICTIONS ON THE BASIS OF MUTUAL INFORMATION**
     * **RANDOM FOREST CLASSIFIER**
     * **LIGHT GBM CLASSIFIER**
     
* **CONCLUSIONS**


   


<html>
    <p style='color:green;'><b>IMPORTING LIBRARIES</b></p>
</html>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from colorama import Fore,Back,Style

<html>
    <h1 style='color:pink;'>ANALYSIS</h1>
</html>

* **METRICS WE WILL FOCUS ON**
  * **F1 SCORE**
  * **PRECISION AND RECALL**

In [ ]:
#Importing Data
df=pd.read_csv("../input/fetal-health-classification/fetal_health.csv")
df.head()

**WE HAVE 2126 ROWS and 22 COLUMNS**

* **THREE TARGET FEATURES**
  * **NORMAL**
  * **SUSPECT**
  * **PATHOLOGICAL**

In [ ]:
print("SHAPE OF OUR DATA IS : ",df.shape)
print(" ")
print("***** DTYPES IN OUR DATA *****")
print(df.dtypes)
      

In [ ]:
#COLUMN NAMES
print(list(df.columns))

In [ ]:
df.describe()

<html>
    <p style='color:green;'><b>CHECKING NULL VALUES</b></p>
</html>

* **There is no null values in this data**

In [ ]:
df.isnull().sum()

<html>
    <p style='color:green;'><b>REMOVING DUPLICATES</b></p>
</html>


In [ ]:
df_dup=df.copy()
df_dup.drop_duplicates(inplace=True)
print("NEW SHAPE AFTER REMOVING DUPLICATES : ",df_dup.shape)
df_dup.head()


In [ ]:
def with_hue(data,feature,ax):
    
    #Number of categories
    num_of_cat=len([x for x in data[feature].unique() if x==x])
    
    bars=ax.patches
    
    for ind in range(num_of_cat):
        ##     Get every hue bar
        ##     ex. 8 X categories, 4 hues =>
        ##    [0, 8, 16, 24] are hue bars for 1st X category
        hueBars=bars[ind:][::num_of_cat] 
        # Get the total height (for percentages)
        total=sum([x.get_height() for x in hueBars])
        #Printing percentages on bar
        for bar in hueBars:
            percentage='{:.1f}%'.format(100 * bar.get_height()/total)
            ax.text(bar.get_x()+bar.get_width()/2.0,
                   bar.get_height(),
                   percentage,
                    ha="center",va="bottom",fontweight='bold',fontsize=15)
    

    
def without_hue(data,feature,ax):
    
    total=float(len(data))
    bars_plot=ax.patches
    
    for bars in bars_plot:
        percentage = '{:.1f}%'.format(100 * bars.get_height()/total)
        x = bars.get_x() + bars.get_width()/2.0
        y = bars.get_height()
        ax.text(x, y,(percentage,bars.get_height()),ha='center',fontweight='bold',fontsize=15)

In [ ]:
sns.set_theme(context='notebook',style='white')
plt.figure(figsize=(20,10))
plt.text(0.7,1100,"Data is imbalanced between three classes",fontweight='bold',fontsize=25,fontstyle='oblique')
plt.title("Countplot of fetal_health",fontweight='bold',fontsize=25,fontstyle='oblique')
ax=sns.countplot(data=df_dup,x="fetal_health",palette='rocket')
plt.xticks(fontweight='bold',fontsize=15)
plt.yticks(fontweight='bold',fontsize=15)
plt.xlabel("fetal_health",fontweight='bold',fontsize=20,fontstyle='oblique')
plt.ylabel("fetal_health",fontweight='bold',fontsize=20,fontstyle='oblique')
without_hue(df_dup,"fetal_health",ax)

**WE HAVE 22 FEATURES AND WE WON'T ANALYSE EACH AND EVERY COLUMN , SO WHAT WE ARE GOING TO DO?**

* **FIRST OF ALL WE WILL DO PREDCITIONS WITHOUT REMOVING ANY FEATURES AND OBSERVE OUR RESULTS, I WILL USE ENSEMBLING METHODS TO OBTAIN RESULTS (RANDOM FOREST AND LIGHT GBM)**

* **SECOND , WE WILL ANALYSE WHICH ARE THE MOST IMPORTANT FEATURES ACCORDING TO OUR TARGET "fetal_health" (THIS TIME WILL TRY SOMETHING NEW , YEAH I LOVE TO DO EXPERIMENTS)**

* **THEN WE WILL REMOVE THOSE FEATURES WHICH ARE LEAST IMPORTANT AND NOT EFFECTIVELY CONTRIBUTING TOWARDS TARGET , WE WILL ALSO CHECK WHETHER LEAST CONTRIBUTING TARGETS ARE INTERACTING FEATURES BETWEEN ANY TWO FEATURES OR NOT AND DECIDE WHETHER TO KEEP THOSE FEATURES OR NOT**

* **WE WIL BE USING "MUTUAL INFORMATION" RATHER THAN CORRELATION TO GET MOST IMPORTANT FEATURES**

* **WELL I AM ASLO LEARNING NEW THINGS :) , IF YOU FIND ANY MISTAKES PLEASE LET ME KNOW AND I WILL TRY MY BEST TO MAKE YOU UNDERSTAND**


In [ ]:
plt.figure(figsize=(20,20))
plt.title("HEATMAP OF DATA",fontsize=25,fontweight='bold')
sns.heatmap(df_dup.corr(),cmap="vlag",annot=True,annot_kws={'size':13})
plt.yticks(fontweight='bold',fontsize=20)
plt.xticks(fontweight='bold',fontsize=20)
plt.show()

<html>
    <p style='color:pink;'><b>IMPORTING LIBRARIES FOR PREDICTION</b></p>
</html>

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
import optuna
from sklearn.metrics import f1_score, roc_auc_score,accuracy_score,confusion_matrix,precision_recall_curve, auc, roc_curve, recall_score, classification_report,plot_confusion_matrix,precision_score

<html>
    <h1 style='color:green;'>PREDICTING WITHOUT REMOVING ANY FEATURES</h1>
</html>

In [ ]:
df_dup_ori=df_dup.copy()
#df_dup_ori=df_dup_ori.drop(['histogram_mode','histogram_median'],axis=1)
df_dup_ori

In [ ]:
X_ori=df_dup_ori.copy()
Y_ori=X_ori.pop('fetal_health')

In [ ]:
Y_ori.value_counts() #Checking for value counts in our targets

In [ ]:
#Splitting the data into train and test sets
x_train_ori,x_test_ori,y_train_ori,y_test_ori=train_test_split(X_ori,Y_ori,test_size=0.3,random_state=42)

<html>
    <p style='color:pink;'><b>RANDOMFOREST CLASSIFIER</b></p>
</html>

<html>
    <p style='color:green;'><b>HYPERPARAMETER TUNING USING OPTUNA</b></p>
</html>

In [ ]:
def objective_rf_ori(trial):
    
    n_estimators = trial.suggest_int('n_estimators', 2, 200)
    max_depth = int(trial.suggest_int('max_depth', 1, 40))
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,class_weight='balanced')
    return cross_val_score(clf, x_train_ori, y_train_ori, 
           n_jobs=-1, cv=5,scoring="f1_micro").mean()

In [ ]:
study=optuna.create_study(direction='maximize',study_name="RANDOM FOREST")
study.optimize(objective_rf_ori,n_trials=50)

In [ ]:
trial_ori=study.best_trial
print("Best accuracy  : ",trial_ori.values)
print("Best Parameter : ",trial_ori.params)

In [ ]:
#Training of the model
model_ori_rf=RandomForestClassifier(n_estimators=158,max_depth=27,class_weight='balanced')
model_ori_rf.fit(x_train_ori,y_train_ori)

In [ ]:
pred_ori_rf=model_ori_rf.predict(x_test_ori)
accuracy_rf=accuracy_score(y_test_ori,pred_ori_rf)
precision_rf=precision_score(y_test_ori,pred_ori_rf,average='weighted')
recall_rf=recall_score(y_test_ori,pred_ori_rf,average='weighted')
f1_rf=f1_score(y_test_ori,pred_ori_rf,average='micro')

print("******* RANDOM FOREST CLASSIFIER RESULTS ********")
print("Accuracy  :", accuracy_rf)
print("Precision : ", precision_rf)
print("Recall    : ",recall_rf)
print("F1 Score  : ",f1_rf)

In [ ]:
print('** CLASSIFICATION REPORT OF RANDOM FOREST CLASSIFIER **')
print(" ")
print(classification_report(y_test_ori,pred_ori_rf))

In [ ]:
y_test_ori.value_counts()

In [ ]:
plot_confusion_matrix(model_ori_rf,x_test_ori,y_test_ori)

<html>
    <p style='color:pink;'><b>LIGHT GBM CLASSIFIER</b></p>
</html>

In [ ]:
import lightgbm as lgb

In [ ]:
def objective_lgbm(trial):
    
    n_estimators = trial.suggest_int('n_estimators', 2, 300)
    max_depth = int(trial.suggest_int('max_depth', 2, 50))
    learning_rate=trial.suggest_loguniform('learning_rate',0.001,1)
    colsample_bytree=trial.suggest_loguniform("colsample_bytree",0.1, 1)
    num_leaves=trial.suggest_int('num_leaves',10,300)
    reg_alpha= trial.suggest_loguniform('reg_alpha',0.1,1)
    reg_lambda= trial.suggest_loguniform('reg_lambda',0.1,1)
    min_split_gain=trial.suggest_loguniform('min_split_gain',0.1,1)
    subsample=trial.suggest_loguniform('subsample',0.1,1)    
    clf = lgb.LGBMClassifier(n_estimators=n_estimators, max_depth=max_depth,
                            learning_rate=learning_rate,colsample_bytree=colsample_bytree,
                            num_leaves=num_leaves,reg_alpha=reg_alpha,reg_lambda=reg_lambda,
                            min_split_gain=min_split_gain,subsample=subsample,class_weight='balanced')
    
    return cross_val_score(clf, x_train_ori, y_train_ori, 
           n_jobs=-1, cv=5).mean()

In [ ]:
study=optuna.create_study(direction='maximize',study_name='LIGHTGBM')
study.optimize(objective_lgbm,n_trials=50)

In [ ]:
trial=study.best_trial
print("Best F1   :", trial.values)
print("Best parameters :",trial.params)

In [ ]:
model_lgbm=lgb.LGBMClassifier(n_estimators=92, max_depth=6, learning_rate=0.15906569428502207, 
                              colsample_bytree=0.9814279100755071, num_leaves=127, reg_alpha=0.16318429872212306, 
                              reg_lambda=0.10256590944837159, 
                              min_split_gain=0.10060819627698767,subsample=0.5115021752468786,class_weight='balanced')

In [ ]:
model_lgbm.fit(x_train_ori,y_train_ori)

In [ ]:
pred_lgbm=model_lgbm.predict(x_test_ori)
accuracy_lgbm=accuracy_score(y_test_ori,pred_lgbm)
precision_lgbm=precision_score(y_test_ori,pred_lgbm,average='weighted')
recall_lgbm=recall_score(y_test_ori,pred_lgbm,average='weighted')
f1_lgbm=f1_score(y_test_ori,pred_lgbm,average='micro')

print("******* LGBM CLASSIFIER RESULTS ********")
print("Accuracy  :" ,  accuracy_lgbm)
print("Precision : ", precision_lgbm)
print("Recall    : ", recall_lgbm)
print("F1 Score  : ", f1_lgbm)

In [ ]:
print('** CLASSIFICATION REPORT OF LIGHTGBM CLASSIFIER **')
print(" ")
print(classification_report(y_test_ori,pred_lgbm))

In [ ]:
plot_confusion_matrix(model_lgbm,x_test_ori,y_test_ori)

**CONCLUSION**
* **LGBM CLASSIFIER PROVIDES US EXTREMELY GOOD RESULTS AS COMPARED TO RANDOM FOREST CLASSIIFIER**
* **NOW THE QUESTION ARISES WHAT IF WE REMOVE SOME FEATURES BASED ON "MUTUAL INFORMATION" AND OBSERVE THE RESULTS , LET'S START WITH THIS EXPERIMENT AND OBSERVE WILL IT MAKE MORE IMPACT?**


<html>
    <h1 style='color:pink;'>ANALYSING WITH "MUTUAL INFORMATION"</h1>
</html>

In [ ]:
df['fetal_health'].value_counts()

In [ ]:
#Importing mutual information library from sklearn feature selection
from sklearn.feature_selection import mutual_info_classif,mutual_info_regression

In [ ]:
X=df_dup.copy()
Y=X.pop("fetal_health")

#ENCODING CATEGORICAL FEATURES
'''for columns in X.select_dtypes('object'):
    X[columns],_=X[columns].factorize()'''

#Evaluating Mutual information score for each feature    
def make_mi_scores(X,Y):
    mi_scores=mutual_info_classif(X,Y)
    mi_scores=pd.Series(mi_scores ,name="MI_scores",index=X.columns)
    mi_scores=mi_scores.sort_values(ascending=False)
    return(mi_scores)

mi_scores=make_mi_scores(X,Y)
mi_scores
    

**PLOTTING MI SCORES**

In [ ]:
#Plotting mutual information bar graph 
def plot_mi_scores(scores):
    scores=scores.sort_values(ascending=True)
    width=np.arange(len(scores))
    ticks=scores.index
    plt.barh(width,scores)
    plt.yticks(width,ticks,fontweight='bold',fontsize=20)
    plt.xticks(fontweight='bold',fontsize=20)
plt.figure(figsize=(16,18))
plot_mi_scores(mi_scores)

**FROM THE ABOVE BAR GRAPH WE WILL TAKE THOSE FEATURES WHOSE MI SCORES ARE NEAR TO 0.0**

* **THERE IS SOMETHING I WANT TO SHARE IS THAT:**

    1. **'histogram_mean' , 'histogram_mode' , 'histogram_median' features are highly correlated with       each other, these features will lead to the problem of multicollinearity,but we should consider       this problem while using regression models , here we will use ensemble methods so this               multicollinearity won't effect much**

In [ ]:
#WE CAN SEE HOW MUCH 'histogram_mean' , 'histogram_mode' , 'histogram_median' features are highly correlated
f,ax=plt.subplots(nrows=1,ncols=3,figsize=(25,10))
sns.regplot(data=df_dup,x="histogram_mean",y="histogram_mode",ax=ax[0])
sns.regplot(data=df_dup,x="histogram_mean",y="histogram_median",ax=ax[1])
sns.regplot(data=df_dup,x="histogram_median",y="histogram_mode",ax=ax[2])

<html>
    <p style='color:green;'><b>VISUALIZING IMPORTANT FEATURES</b></p>
</html>

* **AS WE CAN SEE BELOW IN BOX PLOTS THERE ARE LOT OF OUTLIERS PRESENT IN THE DATA FOR MANY FEATURES IF WE REMOVE THEM THERE WILL LOSS OF DATA , SO WE WILL MOVE FORWARD WITHOUT REMOVING OUTLIERS**

In [ ]:
#VISUALIZING CHOSEN IMPORTANT W.R.T TARGET FEATURE
f,ax1=plt.subplots(nrows=5,ncols=2,figsize=(18,40))

sns.stripplot(data=df_dup,x='fetal_health',y='mean_value_of_short_term_variability',palette='cool',ax=ax1[0][0])
sns.boxplot(data=df_dup,x='fetal_health',y='mean_value_of_short_term_variability',palette='gnuplot',ax=ax1[0][1])

sns.stripplot(data=df_dup,x='fetal_health',y='abnormal_short_term_variability',palette='cool',ax=ax1[1][0])
sns.boxplot(data=df_dup,x='fetal_health',y='abnormal_short_term_variability',palette='gnuplot',ax=ax1[1][1])

sns.stripplot(data=df_dup,x='fetal_health',y='percentage_of_time_with_abnormal_long_term_variability',palette='cool',ax=ax1[2][0])
sns.boxplot(data=df_dup,x='fetal_health',y='percentage_of_time_with_abnormal_long_term_variability',palette='gnuplot',ax=ax1[2][1])

sns.stripplot(data=df_dup,x='fetal_health',y='histogram_mean',palette='cool',ax=ax1[3][0])
sns.boxplot(data=df_dup,x='fetal_health',y='histogram_mean',palette='gnuplot',ax=ax1[3][1])

sns.stripplot(data=df_dup,x='fetal_health',y='histogram_min',palette='cool',ax=ax1[4][0])
sns.boxplot(data=df_dup,x='fetal_health',y='histogram_min',palette='gnuplot',ax=ax1[4][1])


In [ ]:
f,ax1=plt.subplots(nrows=4,ncols=2,figsize=(18,40))

sns.stripplot(data=df_dup,x='fetal_health',y='histogram_variance',palette='cool',ax=ax1[0][0])
sns.boxplot(data=df_dup,x='fetal_health',y='histogram_variance',palette='gnuplot',ax=ax1[0][1])

sns.stripplot(data=df_dup,x='fetal_health',y='accelerations',palette='cool',ax=ax1[1][0])
sns.boxplot(data=df_dup,x='fetal_health',y='accelerations',palette='gnuplot',ax=ax1[1][1])

sns.stripplot(data=df_dup,x='fetal_health',y='histogram_width',palette='cool',ax=ax1[2][0])
sns.boxplot(data=df_dup,x='fetal_health',y='histogram_width',palette='gnuplot',ax=ax1[2][1])

sns.stripplot(data=df_dup,x='fetal_health',y='baseline value',palette='cool',ax=ax1[3][0])
sns.boxplot(data=df_dup,x='fetal_health',y='baseline value',palette='gnuplot',ax=ax1[3][1])


In [ ]:
f,ax1=plt.subplots(nrows=6,ncols=2,figsize=(18,40))

sns.stripplot(data=df_dup,x='fetal_health',y='mean_value_of_long_term_variability',palette='cool',ax=ax1[0][0])
sns.boxplot(data=df_dup,x='fetal_health',y='mean_value_of_long_term_variability',palette='gnuplot',ax=ax1[0][1])

sns.stripplot(data=df_dup,x='fetal_health',y='prolongued_decelerations',palette='cool',ax=ax1[1][0])
sns.boxplot(data=df_dup,x='fetal_health',y='prolongued_decelerations',palette='gnuplot',ax=ax1[1][1])

sns.stripplot(data=df_dup,x='fetal_health',y='fetal_movement',palette='cool',ax=ax1[2][0])
sns.boxplot(data=df_dup,x='fetal_health',y='fetal_movement',palette='gnuplot',ax=ax1[2][1])

sns.stripplot(data=df_dup,x='fetal_health',y='uterine_contractions',palette='cool',ax=ax1[3][0])
sns.boxplot(data=df_dup,x='fetal_health',y='uterine_contractions',palette='gnuplot',ax=ax1[3][1])

sns.stripplot(data=df_dup,x='fetal_health',y='light_decelerations',palette='cool',ax=ax1[4][0])
sns.boxplot(data=df_dup,x='fetal_health',y='light_decelerations',palette='gnuplot',ax=ax1[4][1])

sns.stripplot(data=df_dup,x='fetal_health',y='histogram_tendency',palette='cool',ax=ax1[5][0])
sns.boxplot(data=df_dup,x='fetal_health',y='histogram_tendency',palette='gnuplot',ax=ax1[5][1])



<html>
    <p style='color:pink;'><b>SOME PLOTS THAT MIGHT BE USEFUL</b></p>
</html>


In [ ]:
sns.lmplot(data=df_dup,x='abnormal_short_term_variability',y='fetal_movement',palette='cool'
           ,hue='fetal_health')

In [ ]:
sns.lmplot(data=df_dup,x='mean_value_of_short_term_variability',y='fetal_movement',palette='cool'
           ,hue='fetal_health')


In [ ]:
sns.lmplot(data=df_dup,x='prolongued_decelerations',y='fetal_movement',palette='cool'
           ,hue='fetal_health')


In [ ]:
sns.lmplot(data=df_dup,x='accelerations',y='fetal_movement',palette='cool'
           ,hue='fetal_health')

In [ ]:
df_dup.head()

<html>
    <p style='color:pink;'><b>GETTING IMPORTANT FEATURES INTO DATAFRAME</b></p>
</html>


In [ ]:
mi_scores1=pd.DataFrame(mi_scores)
mi_scores2=pd.DataFrame({'Features':mi_scores1.index,'MI':mi_scores1.MI_scores})
mi_scores2.reset_index(drop=True,inplace=True)
mi_scores2=mi_scores2[mi_scores2["MI"]>0.0100]

mi_scores2

In [ ]:
#CREATING NEW DATAFRAME
df_dup_new=pd.DataFrame()

for i in mi_scores2.Features:
    df_dup_new[i]=df_dup[i]
    
df_dup_new

In [ ]:
#PLOTTING CORRELATION 
df_heatmap=df_dup_new.copy()
df_heatmap['Fetal_health']=Y
plt.figure(figsize=(18,18))
sns.heatmap(data=df_heatmap.corr(),annot=True,cmap='vlag',annot_kws={"size":14})
plt.yticks(fontweight='bold',fontsize=20)
plt.xticks(fontweight='bold',fontsize=20)
plt.show()


<html>
    <p style='color:pink;'><b>LET'S DO PREDICTIONS AND SEE HOW OUR MUTUAL INFORMATION METHOD WORKS</b></p>
</html>


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(df_dup_new,Y,test_size=0.3,random_state=42)

In [ ]:
y_train.value_counts()

In [ ]:
lis=list(x_train.columns)
print(lis)

<html>
    <p style='color:green;'><b>RANDOM FOREST CLASSIFIER</b></p>
</html>


In [ ]:
def objective(trial):
    
    n_estimators = trial.suggest_int('n_estimators', 2, 200)
    max_depth = int(trial.suggest_int('max_depth', 1, 40))
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,class_weight='balanced')
    return cross_val_score(clf, x_train, y_train, 
           n_jobs=-1, cv=5,scoring="f1_micro").mean()

In [ ]:
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=50)

In [ ]:
trial=study.best_trial
print("Best F1   : ",trial.values)
print("Best parameterst: ",trial.params)

In [ ]:
model_rf=RandomForestClassifier(n_estimators=163,max_depth=30,class_weight='balanced')
model_rf.fit(x_train,y_train)

In [ ]:
pred_rf=model_rf.predict(x_test)
accuracy_rf=accuracy_score(y_test,pred_rf)
precision_rf=precision_score(y_test,pred_rf,average='weighted')
recall_rf=recall_score(y_test,pred_rf,average='weighted')
f1_rf=f1_score(y_test,pred_rf,average='micro')

print("******* RANDOM FOREST CLASSIFIER RESULTS ********")
print("Accuracy  :", accuracy_rf)
print("Precision : ", precision_rf)
print("Recall    : ",recall_rf)
print("F1 Score  : ",f1_rf)

In [ ]:
print(classification_report(y_test,pred_rf))

In [ ]:
plot_confusion_matrix(model_rf,x_test,y_test)

<html>
    <p style='color:green;'><b>LIGHT GBM CLASSIFIER</b></p>
</html>


In [ ]:
import lightgbm as lgb

In [ ]:
def objective_lgbm(trial):
    
    n_estimators = trial.suggest_int('n_estimators', 2, 300)
    max_depth = int(trial.suggest_int('max_depth', 2, 50))
    learning_rate=trial.suggest_loguniform('learning_rate',0.001,1)
    colsample_bytree=trial.suggest_loguniform("colsample_bytree",0.1, 1)
    num_leaves=trial.suggest_int('num_leaves',10,300)
    reg_alpha= trial.suggest_loguniform('reg_alpha',0.1,1)
    reg_lambda= trial.suggest_loguniform('reg_lambda',0.1,1)
    min_split_gain=trial.suggest_loguniform('min_split_gain',0.1,1)
    subsample=trial.suggest_loguniform('subsample',0.1,1)    
    clf = lgb.LGBMClassifier(n_estimators=n_estimators, max_depth=max_depth,
                            learning_rate=learning_rate,colsample_bytree=colsample_bytree,
                            num_leaves=num_leaves,reg_alpha=reg_alpha,reg_lambda=reg_lambda,
                            min_split_gain=min_split_gain,subsample=subsample,class_weight='balanced')
    
    return cross_val_score(clf, x_train, y_train, 
           n_jobs=-1, cv=5,scoring='f1_micro').mean()

In [ ]:
study=optuna.create_study(direction='maximize')
study.optimize(objective_lgbm,n_trials=50)

In [ ]:
trial=study.best_trial
print("Best accuarcy : ",trial.values)
print("Best Params   : ",trial.params)

In [ ]:
model_lgbm=lgb.LGBMClassifier(n_estimators=232, max_depth=19, learning_rate=0.6892271434058066, 
                              colsample_bytree=0.7209642378652753, num_leaves=58, reg_alpha=0.2789929636049069, 
                              reg_lambda=0.22306483739443742, 
                              min_split_gain=0.10007706931197725,subsample=0.3982537512515395,class_weight='balanced')

In [ ]:
model_lgbm.fit(x_train,y_train)

In [ ]:
pred_lgbm=model_lgbm.predict(x_test)
accuracy_lgbm=accuracy_score(y_test,pred_lgbm)
precision_lgbm=precision_score(y_test,pred_lgbm,average='weighted')
recall_lgbm=recall_score(y_test,pred_lgbm,average='weighted')
f1_lgbm=f1_score(y_test,pred_lgbm,average='micro')

print("******* LGBM CLASSIFIER RESULTS ********")
print("Accuracy  :" ,  accuracy_lgbm)
print("Precision : ", precision_lgbm)
print("Recall    : ", recall_lgbm)
print("F1 Score  : ", f1_lgbm)

In [ ]:
print(classification_report(y_test,pred_lgbm))

In [ ]:
plot_confusion_matrix(model_lgbm,x_test,y_test)

<html>
    <h1 style='color:pink;'>CONCLUSIONS</h1>
</html>

* **AFTER REMOVING SOME FEATURES THERE IS NO AS SUCH IMPROVEMENT IN THE F1_SCORE**

* **BUT WE ARE ABLE TO GET GOOD SCORE WITH LESS FEATURES , THAT MEANS THE FEATURES WE REMOVED ARE NOT CONTRIBUTING TOWARDS TARGET**

* **LIGHT GBM CLASSIFIER WORKS BETTER THAN RANDOM FOREST CLASSIFIER**

<html>
    <p style='color:green;'><b>WHAT WILL BE THE RESULTS IF WE USE UPSAMPLING</b></p>
</html>


In [ ]:
sm=SMOTE()
x_train_samp,y_train_samp=sm.fit_resample(x_train,y_train)

In [ ]:
def objective(trial):
    
    n_estimators = trial.suggest_int('n_estimators', 2, 200)
    max_depth = int(trial.suggest_int('max_depth', 1, 40))
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
    return cross_val_score(clf, x_train_samp, y_train_samp, 
           n_jobs=-1, cv=5,scoring="f1_micro").mean()

In [ ]:
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=50)

In [ ]:
trial=study.best_trial
print("Best F1   : ",trial.values)
print("Best parameterst: ",trial.params)

In [ ]:
model_rf_samp=RandomForestClassifier(n_estimators=195,max_depth=22)
model_rf_samp.fit(x_train_samp,y_train_samp)

In [ ]:
pred_rf=model_rf_samp.predict(x_test)
accuracy_rf=accuracy_score(y_test,pred_rf)
precision_rf=precision_score(y_test,pred_rf,average='weighted')
recall_rf=recall_score(y_test,pred_rf,average='weighted')
f1_rf=f1_score(y_test,pred_rf,average='micro')

print("******* RANDOM FOREST CLASSIFIER RESULTS WITH UPSAMPLING********")
print("Accuracy  :", accuracy_rf)
print("Precision : ", precision_rf)
print("Recall    : ",recall_rf)
print("F1 Score  : ",f1_rf)

In [ ]:
print(classification_report(y_test,pred_rf))

In [ ]:
plot_confusion_matrix(model_rf_samp,x_test,y_test)

<html>
    <h1 style='color:pink;'><b>FINISH</b></h1>
</html>

* **I WAS TRYING TO DO SOMETHING DIFFERENT ABOVE , BUT IN MY OPINION DATASET AND OUR ENSEMBLERS ARE SO GOOD , THAT EVEN WITHOUT DOING ANYTHING ON DATA WE ARE GETTING GOOD RESULTS**

* **WE EVEN GOT VERY GOOD RESULTS WITH SMOTE UPSAMPLING**

* **IF YOU HAVE ANY DOUBTS OR I HAVE DONE ANY MISTAKE , PLEASE TELL ME I WILL EDIT THIS NOTEBOOK AGAIN , I AM ALWAYS READY TO IMPROVE MYSELF**

* **IF YOU REALLY LIKE MY NOTEBOOOK DO PROVIDE FEEDBACK AND UPVOTE , MAY BE IT WILL BE HELP TO GET A JOB**

* **I LOVE TO DO EXPERIMENTS AND TRYING OUT NEW THINGS , IF YOU WANT ME TO DO SOMETHING LET ME KNOW IN THE COMMENTS SECTION**

* **IF YOU WANT TO KNOW ABOUT MUTUAL INFORMATION PLEASE VISIT FEATURE ENGINEERING MINI COURSE ON KAGGLE AND I HAVE DECIDED TO APPLY THAT IN THIS NOTEBOOK**

* **LET'S MEET ONTO NEXT NOTEBOOK**
